In [1]:
import sys

import pandas as pd

print(sys.executable)  # Path to the Python interpreter
print(sys.version)  # Python version information

/Users/edwardchiu/.local/share/virtualenvs/baha-anime-analysis-rhdRMM96/bin/python
3.12.3 (main, Apr  9 2024, 16:03:47) [Clang 14.0.0 (clang-1400.0.29.202)]


In [2]:
import os

os.chdir('/Users/edwardchiu/Desktop/projects/baha-anime-analysis')
os.getcwd()

'/Users/edwardchiu/Desktop/projects/baha-anime-analysis'

In [27]:
import main.settings as settings
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import json

In [4]:
url = settings.baha_url
user_agent = settings.user_agent
headers = {
    'User-Agent': user_agent
}
res = requests.get(url, headers=headers)
res.status_code

200

In [5]:
soup = BeautifulSoup(res.text, 'html.parser')
soup

<!DOCTYPE html>

<html data-geo="TW" data-theme="light" lang="zh-Hant-TW">
<head>
<title>巴哈姆特動畫瘋</title>
<meta charset="utf-8" content="text/html" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="動畫瘋是由巴哈姆特所推出，正版授權的免費動畫平台，提供與日本同步的當季新番動畫，並含有高達上萬集的過往名作，供動畫迷們自由選擇！" name="description"/>
<meta content="https://i2.bahamut.com.tw/anime/FB_anime.png" name="thumbnail"/>
<meta content="巴哈姆特動畫瘋" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://ani.gamer.com.tw" property="og:url"/>
<meta content="https://i2.bahamut.com.tw/anime/FB_anime.png" property="og:image"/>
<meta content="動畫瘋是由巴哈姆特所推出，正版授權的免費動畫平台，提供與日本同步的當季新番動畫，並含有高達上萬集的過往名作，供動畫迷們自由選擇！" property="og:description"/>
<meta content="app-id=1102650114" name="apple-itunes-app"/>
<link href="https://i2.bahamut.com.tw/favicon.svg?v=1689129528" rel="icon" type="image/svg+xml"/>
<link href="https://ani.gamer.co

In [12]:
newanime_item = soup.select_one('.timeline-ver > .newanime-block')
newanime_item

<div class="newanime-block">
<div class="newanime-date-area new-count-1 fadeIn" data-animesn="113677" data-date-code="1">
<div class="anime-content-block">
<div class="date-block">
<div class="pic-continue">
<img alt="icon-continue" src="https://i2.bahamut.com.tw/anime/pic-continue.svg"/>
</div>
<div class="date-block-border"></div>
</div>
<div class="anime-block">
<a class="anime-card-block" data-gtm="本季新番卡片" href="animeVideo.php?sn=40526">
<div class="anime-pic-block">
<div class="anime-hours-block">
<img alt="clock_img" class="anime-clock-img" src="https://i2.bahamut.com.tw/anime/pic_time_icon.svg"/>
<span class="anime-hours">18:00</span>
</div>
<div class="anime-blocker">

</div>
<div class="line-gradient"></div>
<div class="anime-detail-info-block">
<div class="anime-episode">
<img alt="pic-tv" src="https://i2.bahamut.com.tw/anime/pic-tv.svg"/><p>第2集</p>
</div>
<div class="anime-label-block">
</div>
</div>
<div class="btn-card-block btn-favorite btn-not-active" onclick='animefun.t

In [13]:
newanime_list = newanime_item.select('.newanime-date-area:not(.premium-block)')
len(newanime_list)

53

In [143]:
new_animes = []

# Get the current year
current_year = datetime.now().year

In [144]:
for anime in newanime_list:
    # Anime Name
    name = anime.select_one('.anime-name > p').text.strip()

    # Anime Episode
    episode = anime.select_one('.anime-episode > p').text.strip()

    # Launched date/time
    launched_date = anime.select_one('.anime-date-info').contents[-1].strip().split(' ')[0].replace('/', '-')
    launched_time = anime.select_one('.anime-hours-block > .anime-hours').text.strip()
    launched_at = pd.to_datetime(str(current_year) + '-' + launched_date + ' ' + launched_time).strftime(
        '%Y-%m-%d %H:%M')

    # Cumulative View
    cum_view = anime.select_one('.anime-watch-number > p').text.strip().replace(',', '')
    cum_view = int(float(cum_view.replace('萬', '')) * 10000) if '萬' in cum_view else int(cum_view)

    # Anime episode link
    link = url + anime.select_one('.anime-card-block').get('href')

    # Anime episode thumbnail
    thumbnail = anime.select_one('.anime-blocker > img').get('data-src')

    new_anime = {
        'name': name,
        'episode': episode,
        'launched_at': launched_at,
        'cum_view': cum_view,
        'link': link,
        'thumbnail': thumbnail
    }
    new_animes.append(new_anime)

In [145]:
df_new_animes = pd.DataFrame(new_animes)

In [146]:
df_new_animes

,name,episode,launched_at,cum_view,link,thumbnail
0,青之壬生浪,第2集,2024-10-26 18:00,11000,https://ani.gamer.com.tw/animeVideo.php?sn=40526,https://p2.bahamut.com.tw/B/2KU/90/c6146a391b2...
1,蜻蛉高球,第17集,2024-10-26 09:30,125000,https://ani.gamer.com.tw/animeVideo.php?sn=40426,https://p2.bahamut.com.tw/B/2KU/77/4b06267aba0...
2,超人力霸王雅克,第15.5集,2024-10-26 09:00,46000,https://ani.gamer.com.tw/animeVideo.php?sn=40558,https://p2.bahamut.com.tw/B/2KU/75/43b54fd555a...
3,凍牌~地下麻將鬥牌錄~,第4集,2024-10-26 02:23,71000,https://ani.gamer.com.tw/animeVideo.php?sn=40429,https://p2.bahamut.com.tw/B/2KU/85/3d20ef3e76a...
4,當不成魔法師的女孩,第4集,2024-10-26 01:53,68000,https://ani.gamer.com.tw/animeVideo.php?sn=40431,https://p2.bahamut.com.tw/B/2KU/76/b2df446697f...
5,去參加聯誼，卻發現完全沒有女生在場,第4集,2024-10-26 01:30,172000,https://ani.gamer.com.tw/animeVideo.php?sn=40433,https://p2.bahamut.com.tw/B/2KU/11/c3a18219e90...
6,偶像大師 閃耀色彩 第二季,第4集,2024-10-26 01:00,9724,https://ani.gamer.com.tw/animeVideo.php?sn=40435,https://p2.bahamut.com.tw/B/2KU/47/0d60660e03e...
7,刀劍神域外傳 Gun Gale Online II,第4集,2024-10-26 00:30,305000,https://ani.gamer.com.tw/animeVideo.php?sn=40437,https://p2.bahamut.com.tw/B/2KU/87/f3d7c225101...
8,2.5 次元的誘惑,第17集,2024-10-25 21:30,1336000,https://ani.gamer.com.tw/animeVideo.php?sn=40439,https://p2.bahamut.com.tw/B/2KU/61/606626ffd9e...
9,百姓貴族 2nd Season,第4集,2024-10-25 21:00,92000,https://ani.gamer.com.tw/animeVideo.php?sn=40441,https://p2.bahamut.com.tw/B/2KU/60/eca13a9e296...


In [141]:
def anime_detail(row):
    link = row['link']
    res = requests.get(link, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    # Score value and counts
    score = float(soup.select_one('.score-overall-number').text.strip())
    score_count = int(soup.select_one('.score-overall-people').text.strip().replace(',', '').replace('人評價', ''))

    # Anime types: first launched, director, agent, animator, types
    anime_types = soup.select('.type')
    first_launched_date = pd.to_datetime(anime_types[0].select_one('p').text).strftime('%Y-%m-%d')
    director = anime_types[1].select_one('p').text
    agent = anime_types[2].select_one('p').text
    animator = anime_types[3].select_one('p').text
    types = [typ.text for typ in anime_types[4].select('li')]

    return score, score_count, first_launched_date, director, agent, animator, types

In [148]:
df_new_animes[['score', 'score_count',
               'first_launched_date',
               'director', 'agent',
               'animator', 'types']] = pd.DataFrame(df_new_animes.apply(anime_detail, axis=1).tolist(),
                                                    index=df_new_animes.index)

In [153]:
df_new_animes.sort_values(['cum_view', 'score'], ascending=[False, False], ignore_index=True)

,name,episode,launched_at,cum_view,link,thumbnail,score,score_count,first_launched_date,director,agent,animator,types
0,2.5 次元的誘惑,第17集,2024-10-25 21:30,1336000,https://ani.gamer.com.tw/animeVideo.php?sn=40439,https://p2.bahamut.com.tw/B/2KU/61/606626ffd9e...,4.9,8033,2024-07-05,岡本英樹,木棉花,J.C.STAFF,"[校園, 喜劇, 戀愛, 青春, 職場]"
1,鴨乃橋論的禁忌推理,第16集,2024-10-21 22:30,989000,https://ani.gamer.com.tw/animeVideo.php?sn=40384,https://p2.bahamut.com.tw/B/2KU/22/68be03cbdee...,4.8,3976,2023-10-02,井畑翔太,Ani-One,diomedéa,"[推理, 懸疑]"
2,搖曳露營△ 第三季,第15集,2024-10-23 00:00,922000,https://ani.gamer.com.tw/animeVideo.php?sn=38858,https://p2.bahamut.com.tw/B/2KU/85/12401ad9f5f...,4.9,7361,2024-04-04,登坂晉,Ani-One,8bit,"[溫馨, 悠閒, 料理, 闔家觀賞]"
3,膽大黨,第4集,2024-10-25 00:00,909000,https://ani.gamer.com.tw/animeVideo.php?sn=40452,https://p2.bahamut.com.tw/B/2KU/15/170092c015e...,5.0,14470,2024-10-04,山代風我,木棉花,Science SARU,"[動作, 冒險, 奇幻, 超能力, 科幻, 喜劇, 戀愛, 血腥暴力, 靈異神怪]"
4,Re：從零開始的異世界生活 第三季,第4集,2024-10-23 22:30,782000,https://ani.gamer.com.tw/animeVideo.php?sn=40462,https://p2.bahamut.com.tw/B/2KU/15/f85b1cb6e61...,5.0,10457,2024-10-02,篠原正寬,木棉花,WHITE FOX,"[動作, 冒險, 奇幻, 異世界, 魔法, 時間穿越]"
5,FAIRY TAIL 魔導少年 百年任務,第15集,2024-10-20 17:00,668000,https://ani.gamer.com.tw/animeVideo.php?sn=40546,https://p2.bahamut.com.tw/B/2KU/34/8df7d2ec78c...,4.9,2998,2024-07-07,石平信司、渡部穩寬,木棉花,J.C.STAFF,"[動作, 冒險, 奇幻, 勵志]"
6,嘆氣的亡靈想隱退,第4集,2024-10-20 23:30,604000,https://ani.gamer.com.tw/animeVideo.php?sn=40403,https://p2.bahamut.com.tw/B/2KU/61/90e2fb43bb1...,4.9,5369,2024-09-29,高田真宏,木棉花,ZERO-G,"[冒險, 奇幻, 魔法, 喜劇]"
7,孤單一人的異世界攻略,第5集,2024-10-24 23:00,514000,https://ani.gamer.com.tw/animeVideo.php?sn=40456,https://p2.bahamut.com.tw/B/2KU/03/5f39bd15c21...,4.2,2911,2024-09-26,鹿住朗生,木棉花,Hayabusa Film×Passione、Frontier Engine,"[動作, 冒險, 奇幻, 異世界]"
8,最狂輔助職業【話術士】世界最強戰團聽我號令,第4集,2024-10-21 23:00,498000,https://ani.gamer.com.tw/animeVideo.php?sn=40378,https://p2.bahamut.com.tw/B/2KU/85/524f2ff707d...,4.5,4194,2024-09-30,高村雄太,木棉花,FelixFilm×畫狂,"[動作, 冒險, 奇幻, 魔法]"
9,在地下城尋求邂逅是否搞錯了什麼 第五季,第4集,2024-10-24 21:00,484000,https://ani.gamer.com.tw/animeVideo.php?sn=40458,https://p2.bahamut.com.tw/B/2KU/59/1b14bd5f2e1...,5.0,5191,2024-10-03,橘秀樹,木棉花,J.C.STAFF,"[動作, 冒險, 奇幻, 魔法, 戀愛]"


In [155]:
df_new_animes[df_new_animes['first_launched_date'] < '2024-10-01']

,name,episode,launched_at,cum_view,link,thumbnail,score,score_count,first_launched_date,director,agent,animator,types
1,蜻蛉高球,第17集,2024-10-26 09:30,125000,https://ani.gamer.com.tw/animeVideo.php?sn=40426,https://p2.bahamut.com.tw/B/2KU/77/4b06267aba0...,4.6,743,2024-04-06,オ ジング,回歸線娛樂,OLM,"[勵志, 溫馨, 運動, 競技]"
2,超人力霸王雅克,第15.5集,2024-10-26 09:00,46000,https://ani.gamer.com.tw/animeVideo.php?sn=40558,https://p2.bahamut.com.tw/B/2KU/75/43b54fd555a...,3.9,295,2024-07-06,辻本貴則,野獸國,圓谷製作公司,"[科幻, 特攝, 闔家觀賞, 真人演出]"
8,2.5 次元的誘惑,第17集,2024-10-25 21:30,1336000,https://ani.gamer.com.tw/animeVideo.php?sn=40439,https://p2.bahamut.com.tw/B/2KU/61/606626ffd9e...,4.9,8033,2024-07-05,岡本英樹,木棉花,J.C.STAFF,"[校園, 喜劇, 戀愛, 青春, 職場]"
10,物語系列 第外季＆第怪季,第14集,2024-10-25 19:35,364000,https://ani.gamer.com.tw/animeVideo.php?sn=40258,https://p2.bahamut.com.tw/B/2KU/69/ab2216aaf1c...,4.9,2677,2024-07-06,新房昭之、吉澤翠,曼迪,SHAFT,"[青春, 靈異神怪]"
11,放學後少年花子君,第7集,2024-10-25 12:00,138000,https://ani.gamer.com.tw/animeVideo.php?sn=40523,https://p2.bahamut.com.tw/B/2KU/59/6f08204cf80...,4.9,1391,2023-10-12,北村真咲,Aniplus,Lerche,"[校園, 青春, 悠閒, 靈異神怪, 泡麵番]"
18,孤單一人的異世界攻略,第5集,2024-10-24 23:00,514000,https://ani.gamer.com.tw/animeVideo.php?sn=40456,https://p2.bahamut.com.tw/B/2KU/03/5f39bd15c21...,4.2,2911,2024-09-26,鹿住朗生,木棉花,Hayabusa Film×Passione、Frontier Engine,"[動作, 冒險, 奇幻, 異世界]"
20,TRUMP 系列 Delico’s Nursery,第8集,2024-10-24 01:00,136000,https://ani.gamer.com.tw/animeVideo.php?sn=40469,https://p2.bahamut.com.tw/B/2KU/19/1ac4de0e17f...,4.7,952,2024-08-08,錦織博,曼迪,J.C.STAFF,"[奇幻, 推理, 懸疑]"
22,戰國妖狐 千魔混沌篇,第26集,2024-10-23 23:00,174000,https://ani.gamer.com.tw/animeVideo.php?sn=40528,https://p2.bahamut.com.tw/B/2KU/16/710a2dfe533...,4.8,892,2024-07-03,相澤伽月,Ani-One,WHITE FOX,"[動作, 冒險, 奇幻, 靈異神怪]"
25,搖曳露營△ 第三季,第15集,2024-10-23 00:00,922000,https://ani.gamer.com.tw/animeVideo.php?sn=38858,https://p2.bahamut.com.tw/B/2KU/85/12401ad9f5f...,4.9,7361,2024-04-04,登坂晉,Ani-One,8bit,"[溫馨, 悠閒, 料理, 闔家觀賞]"
29,最狂輔助職業【話術士】世界最強戰團聽我號令,第4集,2024-10-21 23:00,498000,https://ani.gamer.com.tw/animeVideo.php?sn=40378,https://p2.bahamut.com.tw/B/2KU/85/524f2ff707d...,4.5,4194,2024-09-30,高村雄太,木棉花,FelixFilm×畫狂,"[動作, 冒險, 奇幻, 魔法]"
